In [81]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
from tqdm import tqdm
import pandas as pd


In [8]:
path = 'chromedriver.exe'
options = webdriver.ChromeOptions()

In [9]:
chrome_service = Service(executable_path=path)

In [10]:
chrome = webdriver.Chrome(options = options, service = chrome_service)

In [12]:
chrome.get('https://www.nasmm.org/find-a-move-manager/list/?state=pa&country=us')

In [15]:
xpath = '/html/body/div[3]/div/section/div/div[2]/div/ul/li[1]/div[1]/div[2]/h5/b'

In [17]:
chrome.find_element(By.XPATH, xpath).text

'Jill Yesko'

In [18]:
xpth = '/html/body/div[3]/div/section/div/div[2]/div/div[2]/a[6]'

In [20]:
chrome.find_element(By.XPATH, xpth).click()

In [60]:
xpath1 = '/html/body/div[3]/div/section/div/div[2]/div/ul/li[1]'

In [61]:
chrome.find_element(By.XPATH, xpath1).text.split('\n')

['Discover Organizing',
 'Jill Yesko',
 'Bridgeville, PA 15017',
 '',
 'Phone: 412-344-3450',
 'Service Area: Greater Pittsburgh Metro Area',
 'Email: jill@discoverorganizing.com',
 'Website: http://www.discoverorganizing.com',
 'Join Date: 2012',
 'Accredited: 03/2021 — 02/2027',
 'Certified: 01/2023 — 01/2026']

In [62]:
company_name = chrome.find_element(By.XPATH, xpath1).text.split('\n')[0]

In [63]:
name = chrome.find_element(By.XPATH, xpath1).text.split('\n')[1]

In [64]:
address = chrome.find_element(By.XPATH, xpath1).text.split('\n')[2]

In [65]:
phone = chrome.find_element(By.XPATH, xpath1).text.split('\n')[4]

In [66]:
info = chrome.find_element(By.XPATH, xpath1).text.split('\n')

In [67]:
info

['Discover Organizing',
 'Jill Yesko',
 'Bridgeville, PA 15017',
 '',
 'Phone: 412-344-3450',
 'Service Area: Greater Pittsburgh Metro Area',
 'Email: jill@discoverorganizing.com',
 'Website: http://www.discoverorganizing.com',
 'Join Date: 2012',
 'Accredited: 03/2021 — 02/2027',
 'Certified: 01/2023 — 01/2026']

In [71]:
try:
    alternate_phone = [i for i in info if 'Alternative Phone:' in i][0]
except:
    alternate_phone = np.nan

In [74]:
info = chrome.find_element(By.XPATH, xpath1).text.split('\n')

In [76]:
try:
    email = [i for i in info if 'Email:' in i][0]
except:
    email = np.nan

In [ ]:
x = chrome.find_element(By.XPATH,'/html/body/div[3]/div/section/div/div[2]/div/div[1]').text

In [102]:
x1 = int(x.split()[1].split('-')[0])
x2 = int(x.split()[1].split('-')[1])
x3 = int(x.split()[3])

In [105]:
num_pages = x3/(x2-x1+1)

In [106]:
num_people = x2-x1+1

In [ ]:

CN = []
N = []
A = []
PN = []
APN = []
EM = []

chrome.get('https://www.nasmm.org/find-a-move-manager/list/?state=pa&country=us')
x = chrome.find_element(By.XPATH,'/html/body/div[3]/div/section/div/div[2]/div/div[1]').text

x1 = int(x.split()[1].split('-')[0])
x2 = int(x.split()[1].split('-')[1])
x3 = int(x.split()[3])
num_pages = x3/(x2-x1+1)
num_people = x2-x1+1

for k in range(1,num_pages+1):
    chrome.get('https://www.nasmm.org/find-a-move-manager/list/?state=pa&country=us')
    x = chrome.find_element(By.XPATH,'/html/body/div[3]/div/section/div/div[2]/div/div[1]').text
    x1 = int(x.split()[1].split('-')[0])
    x2 = int(x.split()[1].split('-')[1])
    x3 = int(x.split()[3])
    num_people = x2-x1+1
    
    for j in tqdm(range(1,num_people+1)):
        xpath1 = '/html/body/div[3]/div/section/div/div[2]/div/ul/li[{}]'.format(j+(k-1)*10)
        company_name = chrome.find_element(By.XPATH, xpath1).text.split('\n')[0]
        name = chrome.find_element(By.XPATH, xpath1).text.split('\n')[1]
        address = chrome.find_element(By.XPATH, xpath1).text.split('\n')[2]
        phone = chrome.find_element(By.XPATH, xpath1).text.split('\n')[4]
        info = chrome.find_element(By.XPATH, xpath1).text.split('\n')
        try:
            alternate_phone = [i for i in info if 'Alternative Phone:' in i][0]
        except:
            alternate_phone = np.nan
        try:
            email = [i for i in info if 'Email:' in i][0]
        except:
            email = np.nan
        CN.append(company_name)
        N.append(name)
        A.append(address)
        PN.append(phone)
        APN.append(alternate_phone)
        EM.append(email)
    chrome.find_element(By.XPATH,'/html/body/div[3]/div/section/div/div[2]/div/div[2]/a[6]')

100%|██████████| 10/10 [00:01<00:00,  5.72it/s]


In [80]:
N

['Jill Yesko',
 'Jill Kearney, SMM-C',
 'Karen Fretz',
 'Ron Shuma, SMM-C',
 'Connie Corl',
 'Charlotte Schutzman, SMM-C',
 'Roberta Reilly',
 'Erin Covolesky',
 'Mary Ann Kleschick',
 'Justin Showalter']

In [85]:
df = pd.DataFrame(CN, columns = ['Company Name'])
df['Name'] = N
df['Email'] = EM
df['Phone'] = PN
df['Alternative Phone'] = APN
df['Address'] = A

In [86]:
df

,Company Name,Name,Email,Phone,Alternative Phone,Address
0,Discover Organizing,Jill Yesko,Email: jill@discoverorganizing.com,Phone: 412-344-3450,NaN,"Bridgeville, PA 15017"
1,"Specialty Moves by Design, Div of The JDK Grou...","Jill Kearney, SMM-C",Email: info@specialtymovesbydesign.com,Phone: 610-295-0036,NaN,"Allentown, PA 18103"
2,A Thoughtful Transition,Karen Fretz,Email: Karenfretz@athoughtfultransition.com,Phone: 215-660-5052,Alternative Phone: 610-751-3738,"Souderton, PA 18964"
3,A+ Transitions,"Ron Shuma, SMM-C",Email: organize@aplusorganizing.com,Phone: 646-283-5575,NaN,"Pittsburgh, PA 15217"
4,"Adult Transitional Care,LLC",Connie Corl,Email: Conniec@adulttransitionalcare.com,,NaN,Rose Mwangi
5,All Moving Matters,"Charlotte Schutzman, SMM-C",Email: Charlotte@AllMovingMatters.com,Phone: 610-668-2289,Alternative Phone: 610-731-5939,"Gladwyne, PA 19035"
6,Almost Home Relocation Services,Roberta Reilly,Email: rsreilly8@comcast.net,Phone: 484-467-5300,NaN,"West Chester, PA 19380"
7,Amplo Organizing LLC,Erin Covolesky,Email: erin@amploorganizing.com,Phone: 484-574-5672,NaN,"Gilbertsville, PA 19525"
8,Arrive Move Managers,Mary Ann Kleschick,Email: maryann@arrivemovemanagers.com,,Alternative Phone: 2155651060,Marcus Timmons
9,Beyond the Fork in the Road,Justin Showalter,Email: liz@beyondtheforkintheroad.com,Phone: 717-335-8080,NaN,"Denver, PA 17517"
